In [84]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
import re
import nltk
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

## Pre-process the dataset

In [2]:
# Check the length of dataset
count = 0
with open ("Books_5.json") as Train_json:
    for i in Train_json:
        count += 1
print ("Total number of reviews is {}".format(count))

Total number of reviews is 8898041


In [3]:
## Check the first example in the dataset
with open ("Books_5.json") as Train_json:
    first_review = next(Train_json)
    first_review = json.loads(first_review)
    print (first_review)
    print ("\n")
    print (type(first_review))

{'reviewerID': 'A10000012B7CGYKOMPQ4L', 'asin': '000100039X', 'reviewerName': 'Adam', 'helpful': [0, 0], 'reviewText': 'Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are!', 'overall': 5.0, 'summary': 'Wonderful!', 'unixReviewTime': 1355616000, 'reviewTime': '12 16, 2012'}


<class 'dict'>


In [4]:
## Store the usefull information from this huge dataset
dataset = {}
dataset["review_text"] = []
dataset['rating'] = []
count = 0
with open ('Books_5.json') as Train_json:
    for i in Train_json:
        count+=1
        if count % 20 == 0:
            item = json.loads(i)
            dataset["review_text"].append(item["reviewText"])
            dataset["rating"].append(item["overall"])                        

In [5]:
## Check the length
print(len(dataset["review_text"]))
print (len(dataset['rating']))

444902
444902


In [20]:
## Store the review and rating seperately

dataset["review_text_1"] = []
dataset["review_text_2"] = []
dataset["review_text_3"] = []
dataset["review_text_4"] = []
dataset["review_text_5"] = []
for i in range(len(dataset['rating'])):
    if i%1000 == 0:
        print ("finish {} out of 444902".format(i))
    if dataset["rating"][i] == 5.0:
        dataset["review_text_5"].append(dataset['review_text'][i])
    elif dataset["rating"][i] == 4.0:
        dataset["review_text_4"].append(dataset['review_text'][i])
    elif dataset["rating"][i] == 3.0:
        dataset["review_text_3"].append(dataset['review_text'][i])
    elif dataset["rating"][i] == 2.0:
        dataset["review_text_2"].append(dataset['review_text'][i])
    else:
        dataset["review_text_1"].append(dataset['review_text'][i])        
     

finish 0 out of 444902
finish 1000 out of 444902
finish 2000 out of 444902
finish 3000 out of 444902
finish 4000 out of 444902
finish 5000 out of 444902
finish 6000 out of 444902
finish 7000 out of 444902
finish 8000 out of 444902
finish 9000 out of 444902
finish 10000 out of 444902
finish 11000 out of 444902
finish 12000 out of 444902
finish 13000 out of 444902
finish 14000 out of 444902
finish 15000 out of 444902
finish 16000 out of 444902
finish 17000 out of 444902
finish 18000 out of 444902
finish 19000 out of 444902
finish 20000 out of 444902
finish 21000 out of 444902
finish 22000 out of 444902
finish 23000 out of 444902
finish 24000 out of 444902
finish 25000 out of 444902
finish 26000 out of 444902
finish 27000 out of 444902
finish 28000 out of 444902
finish 29000 out of 444902
finish 30000 out of 444902
finish 31000 out of 444902
finish 32000 out of 444902
finish 33000 out of 444902
finish 34000 out of 444902
finish 35000 out of 444902
finish 36000 out of 444902
finish 37000 o

In [21]:
print (len(dataset["review_text_5"]))
print (len(dataset["review_text_4"]))
print (len(dataset["review_text_3"]))
print (len(dataset["review_text_2"]))
print (len(dataset["review_text_1"]))

249099
111070
47619
20931
16183


In [22]:
del(dataset["review_text_1"])
del(dataset["review_text_2"])
del(dataset["review_text_3"])
del(dataset["review_text_4"])
del(dataset["review_text_5"])

In [24]:
## Convert Dataset to dataframe
dataset_df = pd.DataFrame(dataset)
print(len(dataset_df))
dataset_df.head(10)

444902


,rating,review_text
0,5.0,"This book is everything that is simple, delica..."
1,5.0,When I first started writing poetry at age 12 ...
2,5.0,"Khalil Gibran's book, The Prophet, has the pow..."
3,5.0,I was given this book by a writer friend who c...
4,5.0,A book to be treasured. A tremendous poet deal...
5,5.0,Needed to read this and could not find my old ...
6,5.0,I can't think of a title fitting enough. What ...
7,1.0,This book was a philosophical touchstone for i...
8,3.0,This book is tagged as THE PROPHET by KAHLIL G...
9,5.0,Book offers amazing Philosphy and at the same ...


In [25]:
dataset_df_5 = dataset_df[dataset_df["rating"]==5.0]
dataset_df_4 = dataset_df[dataset_df["rating"]==4.0]
dataset_df_3 = dataset_df[dataset_df["rating"]==3.0]
dataset_df_2 = dataset_df[dataset_df["rating"]==2.0]
dataset_df_1 = dataset_df[dataset_df["rating"]==1.0]

In [26]:
print(len(dataset_df_5))
print(len(dataset_df_4))
print(len(dataset_df_3))
print(len(dataset_df_2))
print(len(dataset_df_1))

249099
111070
47619
20931
16183


In [27]:
dataset_df_1_train = dataset_df_1.iloc[0:5000,]
dataset_df_1_test = dataset_df_1.iloc[5000:6000,]

In [28]:
dataset_df_2_train = dataset_df_2.iloc[0:5000,]
dataset_df_2_test = dataset_df_2.iloc[5000:6000,]

In [29]:
dataset_df_3_train = dataset_df_3.iloc[0:5000,]
dataset_df_3_test = dataset_df_3.iloc[5000:6000,]

In [30]:
dataset_df_4_train = dataset_df_4.iloc[0:5000,]
dataset_df_4_test = dataset_df_4.iloc[5000:6000,]

In [31]:
dataset_df_5_train = dataset_df_5.iloc[0:5000,]
dataset_df_5_test = dataset_df_5.iloc[5000:6000,]

In [32]:
frames_train = [dataset_df_5_train, dataset_df_4_train, dataset_df_3_train,dataset_df_2_train,dataset_df_1_train]
frames_test = [dataset_df_5_test, dataset_df_4_test, dataset_df_3_test,dataset_df_2_test,dataset_df_1_test]

In [34]:
dataset_df_train = pd.concat(frames_train)
dataset_df_test = pd.concat(frames_test)

In [35]:
## Clean the reviews
def data_cleaning (raw_review_str):
    ## remove potential HTML tag
    review_text = BeautifulSoup(raw_review_str).get_text()
    ## remove non-letter 
    review_letter = re.sub("[^a-zA-Z]"," ",review_text)
    ## convert to lower case and split the list
    review_lower_list = review_letter.lower().split()
    ## remove stop words and stemming
    stops_eng = set(stopwords.words("english"))
    stemmer = nltk.wordnet.WordNetLemmatizer()
    #review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list]
    review_nostopwords_stemmed = [stemmer.lemmatize(w) for w in review_lower_list if w not in stops_eng]
    ## return 
    return " ".join(review_nostopwords_stemmed)

In [36]:
dataset_df_train["review_cleaned"] = dataset_df_train["review_text"].apply(data_cleaning)
dataset_df_test["review_cleaned"] = dataset_df_test["review_text"].apply(data_cleaning)

/Users/xinmu/anaconda/envs/py36/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/xinmu/anaconda/envs/py36/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


In [39]:
import os
os.getcwd()

'/Users/xinmu/Documents/Xin_Mu/Amazon_book_review'

In [71]:
dataset_df_train.to_csv(path_or_buf="/Users/xinmu/Documents/Xin_Mu/Amazon_book_review/Amazon_book_review_train.csv",index=True)
dataset_df_test.to_csv(path_or_buf="/Users/xinmu/Documents/Xin_Mu/Amazon_book_review/Amazon_book_review_test.csv",index=True)

In [77]:
train_from_csv = pd.read_csv('Amazon_book_review_train.csv',index_col = 0)
train_from_csv.head()
#train_from_csv.isnull().any()

,rating,review_text,review_cleaned
0,5.0,"This book is everything that is simple, delica...",book everything simple delicate true beautiful...
1,5.0,When I first started writing poetry at age 12 ...,first started writing poetry age encouraged re...
2,5.0,"Khalil Gibran's book, The Prophet, has the pow...",khalil gibran book prophet power transform dar...
3,5.0,I was given this book by a writer friend who c...,given book writer friend called beautiful book...
4,5.0,A book to be treasured. A tremendous poet deal...,book treasured tremendous poet dealing mostly ...


In [78]:
train_from_csv[train_from_csv['review_cleaned'].isnull()]

,rating,review_text,review_cleaned
20160,3.0,NaN,NaN
419,2.0,not for me.,NaN
67063,2.0,NaN,NaN


In [79]:
dataset_df_train.isnull().any()

rating            False
review_text       False
review_cleaned    False
dtype: bool

In [80]:
dataset_df_train.dtypes

rating            float64
review_text        object
review_cleaned     object
dtype: object

In [83]:
dataset_df_train.shape

(25000, 3)

In [82]:
dataset_df_train.loc[[20160,419,67063]]

,rating,review_text,review_cleaned
20160,3.0,,
419,2.0,not for me.,
67063,2.0,,


In [47]:
dataset_df_train.shape

(25000, 3)

In [48]:
train_from_csv.shape

(25000, 3)

In [55]:
print(dataset_df.shape)
dataset_df.head(10)

(444902, 2)


,rating,review_text
0,5.0,"This book is everything that is simple, delica..."
1,5.0,When I first started writing poetry at age 12 ...
2,5.0,"Khalil Gibran's book, The Prophet, has the pow..."
3,5.0,I was given this book by a writer friend who c...
4,5.0,A book to be treasured. A tremendous poet deal...
5,5.0,Needed to read this and could not find my old ...
6,5.0,I can't think of a title fitting enough. What ...
7,1.0,This book was a philosophical touchstone for i...
8,3.0,This book is tagged as THE PROPHET by KAHLIL G...
9,5.0,Book offers amazing Philosphy and at the same ...


In [85]:
## Split the the dataset to training and test data
train_set, test_set = sklearn.model_selection.train_test_split(dataset_df,train_size=0.8)

/Users/xinmu/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [86]:
## Put all the review of train_set in a list to generate the features
clean_train_reviews = []
for i in dataset_df_train["review_cleaned"]:
    clean_train_reviews.append(i)

In [87]:
## Put all the review of test_set in a list 
clean_test_reviews = []
for i in dataset_df_test["review_cleaned"]:
    clean_test_reviews.append(i) 

In [88]:
## Generate Anather set of features
vectorizer = TfidfVectorizer(analyzer = "word", ngram_range=(1, 2), tokenizer = None, preprocessor = None, stop_words = None, max_features=20000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
test_data_features = vectorizer.transform(clean_test_reviews)

In [89]:
train_data_features.shape

(25000, 20000)

In [97]:
train_data_features = train_data_features.toarray() ## train_data_features now is a very sparse matrix
test_data_features = test_data_features.toarray()

In [91]:
type(train_data_features)

numpy.ndarray

In [92]:
train_data_features.shape

(25000, 20000)

In [95]:
forest = RandomForestClassifier(n_estimators = 200) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(train_data_features, dataset_df_train["rating"])

## Apply the model on test data set

In [98]:
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

In [99]:
count_correct = 0
for i in range(len(result)):
    if result[i] == dataset_df_test['rating'].iloc[i]:
        count_correct+=1
print (count_correct/len(result))
    

0.4526


In [100]:
c1 = 0
c2 = 0
c3 = 0
c4 = 0
c5 = 0
for i in range(len(result)):
    if dataset_df_test['rating'].iloc[i] == 1.0:
        c1 += 1
    elif dataset_df_test['rating'].iloc[i] == 2.0:
        c2 += 1
    elif dataset_df_test['rating'].iloc[i] == 3.0:
        c3 += 1
    elif dataset_df_test['rating'].iloc[i] == 4.0:
        c4 += 1
    else:
        c5 += 1
        

In [101]:
print (c1/len(result), c2/len(result), c3/len(result), c4/len(result), c5/len(result))

0.2 0.2 0.2 0.2 0.2


In [102]:
assert(len(result)== len(dataset_df_test))